In [3]:
# =============================================================================
#           04_dinov2_small_inference.ipynb: DINOv2 模型推理与提交
# =============================================================================

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from torchvision import transforms
from tqdm.notebook import tqdm

# -----------------------------------------------------------------------------
# 1. 配置参数
# -----------------------------------------------------------------------------
# !! 请确保这里的文件名与你训练保存的模型文件名一致 !!
MODEL_PATH = 'best_dinov2_small_model.pth' 

IMAGE_DIR = 'csiro-biomass'
TEST_CSV = 'csiro-biomass/test.csv'
BATCH_SIZE = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"正在使用设备: {device}")
print(f"将加载模型: {MODEL_PATH}")

# -----------------------------------------------------------------------------
# 2. 定义模型结构 (DINOv2 VisualModel)
# -----------------------------------------------------------------------------
# 注意：推理时的模型定义必须与训练时完全一致
class DINOv2VisualModel(nn.Module):
    def __init__(self, num_targets=5):
        super(DINOv2VisualModel, self).__init__()
        
        print("正在初始化 DINOv2 架构...")
        # 加载 backbone
        self.backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
        
        # Small 版本的输出维度是 384
        self.embed_dim = 384 
        
        # 定义回归头
        self.head = nn.Sequential(
            nn.Linear(self.embed_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_targets)
        )

    def forward(self, image):
        # 提取特征
        features = self.backbone.forward_features(image)['x_norm_clstoken']
        # 预测
        output = self.head(features)
        return output

# -----------------------------------------------------------------------------
# 3. 定义推理数据集
# -----------------------------------------------------------------------------
class InferenceDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 获取相对路径
        img_path_rel = self.df.iloc[idx]['image_path']
        img_path = os.path.join(self.image_dir, img_path_rel)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            # 容错处理：生成全黑图片
            print(f"警告: 找不到图片 {img_path}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))
            
        if self.transform:
            image = self.transform(image)
            
        return image

# -----------------------------------------------------------------------------
# 4. 准备数据与模型
# -----------------------------------------------------------------------------

# 定义图像变换 (验证集标准)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 读取测试列表
test_df = pd.read_csv(TEST_CSV)

# 去重优化：每张图片只预测一次
unique_images_df = test_df[['image_path']].drop_duplicates().reset_index(drop=True)
print(f"测试集原始行数: {len(test_df)}")
print(f"唯一图片数量: {len(unique_images_df)}")

# 创建加载器
test_dataset = InferenceDataset(unique_images_df, IMAGE_DIR, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# 实例化并加载模型
model = DINOv2VisualModel(num_targets=5).to(device)

if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print("✅ 模型权重加载成功！")
else:
    raise FileNotFoundError(f"❌ 找不到模型文件: {MODEL_PATH}，请检查路径。")

model.eval() # 开启评估模式

# -----------------------------------------------------------------------------
# 5. 执行推理
# -----------------------------------------------------------------------------
print("\n开始推理...")

all_predictions = []
# 目标列顺序 (必须与训练时一致)
target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']

with torch.no_grad():
    for images in tqdm(test_loader, desc="进度"):
        images = images.to(device)
        
        # 1. 模型预测 (Log 尺度)
        log_preds = model(images)
        
        # 2. 还原数值 (Exp 变换)
        preds = np.expm1(log_preds.cpu().numpy())
        
        # 3. 防止负值 (物理上生物量不能为负，虽然expm1通常>=-1，但加个保险)
        preds = np.maximum(preds, 0)
        
        all_predictions.append(preds)

# 合并结果
predictions_array = np.concatenate(all_predictions, axis=0)

# -----------------------------------------------------------------------------
# 6. 生成提交文件
# -----------------------------------------------------------------------------
print("正在生成 submission.csv ...")

# 1. 将预测结果转为 DataFrame
pred_df_wide = pd.DataFrame(predictions_array, columns=target_cols)
# 2. 将结果与 image_path 关联
result_df = pd.concat([unique_images_df, pred_df_wide], axis=1)

# 3. 将结果合并回原始的 test.csv (通过 image_path)
final_df = pd.merge(test_df, result_df, on='image_path', how='left')

# 4. 根据每一行的 target_name 提取对应的预测值
def get_prediction(row):
    t_name = row['target_name']
    if t_name in row:
        return row[t_name]
    else:
        return 0.0

final_df['target'] = final_df.apply(get_prediction, axis=1)

# 5. 保存最终文件
submission_df = final_df[['sample_id', 'target']]
submission_df.to_csv('submission.csv', index=False)

print("="*50)
print("🎉 成功！文件 'submission.csv' 已生成。")
print("前 5 行预览:")
print(submission_df.head())
print("="*50)


正在使用设备: cpu
将加载模型: best_dinov2_small_model.pth
测试集原始行数: 5
唯一图片数量: 1
正在初始化 DINOv2 架构...


Using cache found in /Users/nisikin/.cache/torch/hub/facebookresearch_dinov2_main


✅ 模型权重加载成功！

开始推理...


/var/folders/mw/77ps8s1x14zg2_q9t7f74t4r0000gn/T/ipykernel_63778/418644289.py:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH,

进度:   0%|          | 0/1 [00:00<?, ?it/s]

正在生成 submission.csv ...
🎉 成功！文件 'submission.csv' 已生成。
前 5 行预览:
                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.000184
1    ID1001187975__Dry_Dead_g  17.005959
2   ID1001187975__Dry_Green_g  31.306974
3   ID1001187975__Dry_Total_g  48.736496
4         ID1001187975__GDM_g  27.792139
